# Web Scraping Notebook

---

### Import Modules & Read in Data Frame

In [1]:
import json
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('sqr_no_comments.csv')

---

### Data Base Number (DBN) List

Create a list of all DBN's to pass into end of https://insideschools.org/school/ url. 

In [3]:
dbn_list = list(df.dbn)

Adjust dbn_list range in order to scrape and save in batches. Ran into issues when trying to scrape too much at one time. 

In [5]:
dbn_list = dbn_list[1000:]

---

### Scrape

Use Selenium's headless mode option so browser does not continually open with every school's website. Must set a path to Selenium's downloaded chromedriver in order to function properly. 

In [6]:
# Create an instance of ChromeOptions
options = webdriver.ChromeOptions()
# Run headless mode 
options.add_argument("headless")
# Instatiate chrome driver and pass in the file path to chromedriver
driver = webdriver.Chrome('/Users/kevinmacmat/Desktop/capstone/chromedriver', options=options)

Get comments for past 6 years and output them to output_list. The 6 year cutoff was determined due to the SQR's availability for those years.

In [7]:
output_list = []

In [8]:
for dbn in dbn_list:
    # Get website 
    driver.get('https://insideschools.org/school/' + dbn)
    # Switch to iframe containing script tag
    driver.switch_to.frame(1)
    # Grab the text
    text = driver.page_source
    # Switch out of iframe
    driver.switch_to.default_content()
    # Parse and process the source with BeautifulSoup module by creating an BS object
    soup = BeautifulSoup(text, 'lxml')
    # Access the soup and find the script element's id
    thread = soup.find("script", {"id": "disqus-threadData"})
    # Turn the bs4 tag into a string, remove the script tag, and access the json
    site_json=json.loads(str(thread)[48:-9])
    # Instantiate comments list
    comments_list = []
    # Navigate and loop json, filtering comments by date, to append comments to comments_list
    for comment in site_json['response']['posts']:
        if '2014' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2015' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2016' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2017' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2018' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2019' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2020' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        else:
            continue
    # Append list of comments to output_list
    output_list.append(comments_list)

---

### Make Data Frame

In [10]:
# Create dataframe with list of data base numbers
batch_df = pd.DataFrame(dbn_list, columns=['dbn'])

In [11]:
# Add comments of recently scraped batch of comments to comments column 
batch_df['comments'] = output_list

---

### Convert data frame to CSV and export

In [13]:
batch_df.to_csv('batch_1000-end_comments.csv', index=False)